In [170]:
import numpy as np
import pandas as pd
import datetime

In [171]:
def gamelag(gm):
    yr,wk=(gm[:4],gm[4:])
    if wk=='1':
        return str(int(yr)-1)+'17'
    else:
        return yr+str(int(wk)-1)

In [173]:
main=pd.read_csv("nflsavant/pbp-2014.csv")
for i in range(2015,2020):
    main=main.append(pd.read_csv("nflsavant/pbp-{0}.csv".format(str(i))))
main['Last']=main.Description.str.split(".", expand=True)[1].str.split(" ", expand=True)[0]
main['First']=main.Description.str.split(".", expand=True)[0].str.split("-", expand=True)[1]
main=main.join(main.groupby('SeasonYear')['GameDate'].agg(['min']), on='SeasonYear')
main['GameDate'] = pd.to_datetime(main['GameDate'], format='%Y-%m-%d')
main['min'] = pd.to_datetime(main['min'], format='%Y-%m-%d')
main['Week']=((main['GameDate']-main['min']).astype(str).str.split(" ", expand=True)[0].astype(int)//7)+1

In [174]:
main['GameDate']

0       2014-09-07
1       2014-09-07
2       2014-09-07
3       2014-09-07
4       2014-09-07
           ...    
26361   2019-09-08
26362   2019-09-08
26363   2019-09-05
26364   2019-09-05
26365   2019-09-08
Name: GameDate, Length: 254598, dtype: datetime64[ns]

In [104]:
rush=main[(main['IsRush']==1) & (main['IsNoPlay']==0)]

In [166]:
smcols=['Yards','IsRush','IsTouchdown']
his=30
samp=rush[['Last','First','Week','SeasonYear']+smcols].groupby(['Last','First','Week','SeasonYear']).sum().reset_index()
samp['Game0']=samp['SeasonYear'].astype(str)+samp['Week'].astype(str)
rnm={}
for i in range(1,his+1):
    lagl=[]
    for col in smcols:
        lagl.append(col+str(i))
        rnm[col]=col+str(i)
    rnm['Game0']='Game'+str(i)
    tmp=samp.rename(columns=rnm)
    samp['Game'+str(i)]=samp['Game'+str(i-1)].apply(gamelag)
    samp=samp.merge(tmp[['Game'+str(i),'First','Last']+lagl], how='left', on=['Game'+str(i),'First','Last'])

In [167]:
samp.iloc[5001]

Last             INGRAM
First                 M
Week                  3
SeasonYear         2019
Yards               103
                  ...  
IsTouchdown29         0
Game30            20177
Yards30              74
IsRush30             15
IsTouchdown30         0
Name: 5001, Length: 128, dtype: object

In [117]:
samp[samp['Yards']>200]

,Last,First,Week,SeasonYear,Yards,IsRush,IsTouchdown
123,AJAYI,J,6,2016,204,25,2
149,AJAYI,J,16,2016,206,32,1
738,BELL,L,6,2017,203,32,1
756,BELL,L,11,2014,204,33,1
765,BELL,L,14,2016,236,38,3
2392,CROWELL,I,5,2018,216,14,1
3540,FOURNETTE,L,4,2019,207,22,0
4220,GURLEY,T,6,2018,208,28,2
4440,HENRY,D,14,2018,238,17,4
5452,JOHNSON,K,4,2019,225,26,1


In [169]:
main['DefenseTeam'].value_counts()

NYG    8286
CLE    8233
MIA    8174
KC     8157
CIN    8154
SF     8135
JAX    8101
ARI    8094
PHI    8091
TB     8073
TEN    8072
OAK    8057
DEN    8035
NYJ    8030
LA     8015
GB     7965
ATL    7948
HOU    7886
BUF    7864
CHI    7848
WAS    7847
IND    7842
MIN    7829
SEA    7816
BAL    7799
NO     7795
DET    7783
CAR    7765
NE     7739
DAL    7726
PIT    7709
SD     4251
LAC    3479
Name: DefenseTeam, dtype: int64